# * VINSIGHT : Revenue
    Postpaid Revenue : TMH(Jan-Jun 2024) ใช้ data พี่เป้า(Definition เก่า ตัวเลขใกล้เคียงกัน)
    Postpaid Revenue รวมยอด topping(แต่ยกเว้นจ่ายด้วยวิธีอื่นที่ไม่เข้า bill เช่น TMN)

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
         
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-10-27, 12:31:00

DataFrame: 17290 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview(Corp) by Period

In [4]:
''' Revenue(Corp) Yearly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_yearly_df = rev_corp_yearly_df.loc[rev_corp_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_yearly_df['TOTAL(Corp)'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B0R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['PRE'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB1R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['POST'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB2R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TOL'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB3R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TVS'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB4R000100', rev_corp_yearly_df['C'], 0)

rev_corp_yearly_df = rev_corp_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_yearly_df['YoY'] = rev_corp_yearly_df['TOTAL(Corp)'] - rev_corp_yearly_df['TOTAL(Corp)'].shift()
rev_corp_yearly_df = rev_corp_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_corp_yearly_df = rev_corp_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_yearly_df_display = rev_corp_yearly_df.copy()
rev_corp_yearly_df_display['ACTUAL_AS_OF'] = rev_corp_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_yearly_df_display[col] = rev_corp_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Corp),PRE,POST,TOL,TVS
0,2024,2025-09-22 06:15:44,20241231,0,"149,825,803,306","48,319,930,436","78,295,748,348","18,827,249,425","4,018,244,147"
1,2025,2025-10-27 06:22:35,20251025,"-25,546,761,900","124,279,041,406","40,718,570,904","64,627,395,240","15,747,294,817","2,678,097,252"


In [5]:
''' Revenue(Corp) Quarterly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_quarterly_df = rev_corp_quarterly_df.loc[rev_corp_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_quarterly_df['TOTAL(Corp)'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B0R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['PRE'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB1R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['POST'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB2R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TOL'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB3R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TVS'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB4R000100', rev_corp_quarterly_df['C'], 0)

rev_corp_quarterly_df = rev_corp_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_quarterly_df['QoQ'] = rev_corp_quarterly_df['TOTAL(Corp)'] - rev_corp_quarterly_df['TOTAL(Corp)'].shift()
rev_corp_quarterly_df = rev_corp_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_corp_quarterly_df = rev_corp_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_quarterly_df_display = rev_corp_quarterly_df.copy()
rev_corp_quarterly_df_display['ACTUAL_AS_OF'] = rev_corp_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_quarterly_df_display[col] = rev_corp_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Corp),PRE,POST,TOL,TVS
0,20241,2025-09-22 06:15:44,20240331,0,"37,260,216,440","11,842,176,786","19,753,202,663","4,569,920,004","1,013,212,668"
1,20242,2025-09-22 06:15:44,20240630,"-64,093,390","37,196,123,050","11,858,030,293","19,630,569,282","4,665,781,597","968,583,208"
2,20243,2025-09-22 06:15:44,20240930,"428,367,688","37,624,490,738","12,164,985,004","19,459,112,025","4,803,785,483","1,110,012,430"
3,20244,2025-09-22 06:15:44,20241231,"120,482,340","37,744,973,078","12,454,738,354","19,452,864,379","4,787,762,341","926,435,841"
4,20251,2025-10-27 06:22:35,20250331,"-219,874,986","37,525,098,091","12,378,982,586","19,368,310,563","4,810,187,607","883,137,861"
5,20252,2025-10-27 06:22:35,20250630,"-82,230,437","37,442,867,655","12,360,832,103","19,312,243,921","4,833,931,536","860,149,762"
6,20253,2025-10-27 06:22:35,20250930,"-31,155,799","37,411,711,855","12,529,561,866","19,168,783,896","4,823,602,770","787,190,308"
7,20254,2025-10-27 06:22:35,20251025,"-25,512,348,051","11,899,363,805","3,449,194,348","6,778,056,860","1,279,572,905","147,619,322"


In [6]:
''' Revenue(Corp) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_corp_monthly_df = rev_corp_monthly_df.loc[rev_corp_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_monthly_df['TOTAL(Corp)'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B0R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['PRE'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB1R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['POST'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB2R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TOL'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB3R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TVS'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB4R000100', rev_corp_monthly_df['C'], 0)

rev_corp_monthly_df = rev_corp_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_monthly_df['MoM'] = rev_corp_monthly_df['TOTAL(Corp)'] - rev_corp_monthly_df['TOTAL(Corp)'].shift()
rev_corp_monthly_df = rev_corp_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_corp_monthly_df = rev_corp_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_monthly_df_display = rev_corp_monthly_df.copy()
rev_corp_monthly_df_display['ACTUAL_AS_OF'] = rev_corp_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_monthly_df_display[col] = rev_corp_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Corp),PRE,POST,TOL,TVS
0,202401,2025-09-22 06:15:44,20240131,0,"12,527,775,340","4,014,674,524","6,633,797,001","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,20240229,"-279,732,061","12,248,043,279","3,796,713,842","6,572,934,091","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,20240331,"236,354,542","12,484,397,821","4,030,788,420","6,546,471,571","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,20240430,"-125,805,023","12,358,592,798","3,883,185,086","6,572,828,730","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,20240531,"107,213,556","12,465,806,354","4,042,374,933","6,520,016,738","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,20240630,"-94,082,457","12,371,723,897","3,932,470,274","6,537,723,813","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,20240731,"118,155,942","12,489,879,839","4,072,664,861","6,484,830,524","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,20240831,"134,708,079","12,624,587,918","4,109,958,930","6,483,049,467","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,20240930,"-114,564,937","12,510,022,981","3,982,361,213","6,491,232,034","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,20241031,"59,487,568","12,569,510,549","4,128,174,548","6,485,731,903","1,598,256,224","311,443,243"


In [7]:
''' Revenue(Corp) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_weekly_df = rev_corp_weekly_df.loc[rev_corp_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_weekly_df['TOTAL(Corp)'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B0R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['PRE'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB1R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['POST'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB2R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TOL'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB3R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TVS'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB4R000100', rev_corp_weekly_df['C'], 0)

rev_corp_weekly_df = rev_corp_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_weekly_df['WoW'] = rev_corp_weekly_df['TOTAL(Corp)'] - rev_corp_weekly_df['TOTAL(Corp)'].shift()
rev_corp_weekly_df = rev_corp_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_corp_weekly_df = rev_corp_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_weekly_df_display = rev_corp_weekly_df.copy()
rev_corp_weekly_df_display['ACTUAL_AS_OF'] = rev_corp_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_weekly_df_display[col] = rev_corp_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Corp),PRE,POST,TOL,TVS
0,2025036,2025-10-27 06:22:35,20250907,0,"3,016,126,228","1,029,837,825","1,572,732,624","370,142,141","39,859,455"
1,2025037,2025-10-27 06:22:35,20250914,"-192,092,927","2,824,033,301","901,311,685","1,469,722,888","403,632,978","47,433,979"
2,2025038,2025-10-27 06:22:35,20250921,"-217,070,720","2,606,962,581","904,879,170","1,319,558,305","335,704,856","45,014,748"
3,2025039,2025-10-27 06:22:35,20250928,"584,402,148","3,191,364,729","895,748,085","1,725,446,013","500,497,244","67,582,597"
4,2025040,2025-10-27 06:22:35,20251005,"92,749,624","3,284,114,353","1,104,285,279","1,791,244,214","222,128,759","111,291,646"
5,2025041,2025-10-27 06:22:35,20251012,"-59,476,125","3,224,638,229","939,839,028","1,861,782,271","382,745,354","36,493,754"
6,2025042,2025-10-27 06:22:35,20251019,"874,766,327","4,099,404,556","936,284,717","2,590,374,932","510,648,779","58,724,527"
7,2025043,2025-10-27 06:22:35,20251025,"-2,047,443,734","2,051,960,822","797,790,373","837,149,877","164,050,013","17,352,466"


In [8]:
''' Revenue(Corp) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_daily_df['TOTAL(Corp)'] = np.where(rev_corp_daily_df['METRIC_CD']=='B0R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['PRE'] = np.where(rev_corp_daily_df['METRIC_CD']=='B1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB1R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['POST'] = np.where(rev_corp_daily_df['METRIC_CD']=='B2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB2R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TOL'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB3R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TVS'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB4R000100', rev_corp_daily_df['C'], 0)

rev_corp_daily_df = rev_corp_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_daily_df['DoD'] = rev_corp_daily_df['TOTAL(Corp)'] - rev_corp_daily_df['TOTAL(Corp)'].shift()
rev_corp_daily_df = rev_corp_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_corp_daily_df = rev_corp_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_daily_df_display = rev_corp_daily_df.copy()
mod_col_list = rev_corp_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_daily_df_display[col] = rev_corp_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Corp),PRE,POST,TOL,TVS
0,202510,20251026,2025-10-27 06:22:35,"-210,856,084",0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,"2,473,030","210,856,084","132,898,275",0,0,0
2,202510,20251024,2025-10-27 06:22:35,"8,175,025","208,383,054","130,189,624",0,0,0
3,202510,20251023,2025-10-27 06:22:35,"-789,447,297","200,208,030","122,240,360",0,0,0
4,202510,20251022,2025-10-27 06:22:35,"778,892,854","989,655,326","127,587,642","679,448,028","164,050,013","17,352,466"
5,202510,20251021,2025-10-27 06:22:35,"-21,333,383","210,762,472","131,687,372","78,936,601",0,0
6,202510,20251020,2025-10-27 06:22:35,"-930,800,615","232,095,855","153,187,100","78,765,248",0,0
7,202510,20251019,2025-10-27 06:22:35,"949,518,446","1,162,896,470","136,943,487","841,189,172","166,300,027","17,128,391"
8,202510,20251018,2025-10-27 06:22:35,"-4,030,559","213,378,025","135,110,232","77,984,255",0,0
9,202510,20251017,2025-10-27 06:22:35,"-817,347,009","217,408,584","139,227,959","77,987,663",0,0


## Overview(Geo) by Period

In [9]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df['YoY'] = rev_geo_yearly_df['TOTAL(Geo)'] - rev_geo_yearly_df['TOTAL(Geo)'].shift()
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_yearly_df_display = rev_geo_yearly_df.copy()
rev_geo_yearly_df_display['ACTUAL_AS_OF'] = rev_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df_display[col] = rev_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2024,2025-09-22 06:15:44,20241231,0,"142,125,302,342","48,319,930,436","27,043,194,488","21,276,735,948","70,959,878,334","39,407,414,546","31,552,463,789","18,827,249,425","4,018,244,147"
1,2025,2025-10-27 06:22:35,20251025,"-24,455,421,941","117,669,880,401","40,718,570,904","23,562,207,232","17,156,363,672","58,295,182,090","34,065,354,531","24,460,562,897","15,747,294,817","2,678,097,252"


In [10]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df['QoQ'] = rev_geo_quarterly_df['TOTAL(Geo)'] - rev_geo_quarterly_df['TOTAL(Geo)'].shift()
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_quarterly_df_display = rev_geo_quarterly_df.copy()
rev_geo_quarterly_df_display['ACTUAL_AS_OF'] = rev_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df_display[col] = rev_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,20241,2025-09-22 06:15:44,20240331,0,"35,361,933,475","11,842,176,786","6,519,610,988","5,322,565,798","17,936,624,018","9,850,451,722","8,086,172,296","4,569,920,004","1,013,212,668"
1,20242,2025-09-22 06:15:44,20240630,"-77,400,943","35,284,532,532","11,858,030,293","6,615,472,616","5,242,557,676","17,792,137,434","9,838,450,875","7,953,686,559","4,665,781,597","968,583,208"
2,20243,2025-09-22 06:15:44,20240930,"422,283,505","35,706,816,037","12,164,985,004","6,830,749,822","5,334,235,182","17,628,033,120","9,834,028,205","7,794,004,915","4,803,785,483","1,110,012,430"
3,20244,2025-09-22 06:15:44,20241231,"65,204,262","35,772,020,298","12,454,738,354","7,077,361,062","5,377,377,292","17,603,083,763","9,884,483,743","7,718,600,020","4,787,762,341","926,435,841"
4,20251,2025-10-27 06:22:35,20250331,"-118,951,220","35,653,069,078","12,378,982,586","7,114,703,403","5,264,279,183","17,580,761,024","9,989,547,936","7,591,213,088","4,810,187,607","883,137,861"
5,20252,2025-10-27 06:22:35,20250630,"-109,128,810","35,543,940,268","12,360,832,103","7,131,257,283","5,229,574,820","17,489,026,867","9,958,948,817","7,530,078,051","4,833,931,536","860,149,762"
6,20253,2025-10-27 06:22:35,20250930,"47,504,063","35,591,444,331","12,529,561,866","7,305,120,252","5,224,441,615","17,451,089,386","10,047,721,142","7,403,368,243","4,823,602,770","787,190,308"
7,20254,2025-10-27 06:22:35,20251025,"-24,710,017,606","10,881,426,724","3,449,194,348","2,011,126,294","1,438,068,055","5,774,304,812","4,069,136,636","1,935,903,514","1,279,572,905","147,619,322"


In [11]:
''' Revenue(Geo) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df['MoM'] = rev_geo_monthly_df['TOTAL(Geo)'] - rev_geo_monthly_df['TOTAL(Geo)'].shift()
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_monthly_df_display = rev_geo_monthly_df.copy()
rev_geo_monthly_df_display['ACTUAL_AS_OF'] = rev_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df_display[col] = rev_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-09-22 06:15:44,20240131,0,"11,881,730,399","4,014,674,524","2,211,126,641","1,803,547,883","6,020,820,300","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,20240229,"-255,166,666","11,626,563,733","3,796,713,842","2,082,104,053","1,714,609,789","5,974,961,131","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,20240331,"227,075,609","11,853,639,343","4,030,788,420","2,226,380,294","1,804,408,125","5,940,842,587","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,20240430,"-130,159,939","11,723,479,404","3,883,185,086","2,140,030,223","1,743,154,862","5,961,586,281","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,20240531,"118,388,442","11,841,867,846","4,042,374,933","2,269,268,086","1,773,106,846","5,921,618,672","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,20240630,"-122,682,564","11,719,185,282","3,932,470,274","2,206,174,307","1,726,295,968","5,908,932,481","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,20240731,"138,027,673","11,857,212,955","4,072,664,861","2,272,833,725","1,799,831,136","5,878,727,794","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,20240831,"135,679,548","11,992,892,504","4,109,958,930","2,312,752,271","1,797,206,658","5,880,942,518","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,20240930,"-136,181,926","11,856,710,578","3,982,361,213","2,245,163,825","1,737,197,388","5,868,362,808","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,20241031,"43,673,946","11,900,384,524","4,128,174,548","2,345,529,988","1,782,644,560","5,862,510,509","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [12]:
''' Revenue(Geo) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df['WoW'] = rev_geo_weekly_df['TOTAL(Geo)'] - rev_geo_weekly_df['TOTAL(Geo)'].shift()
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_weekly_df_display = rev_geo_weekly_df.copy()
rev_geo_weekly_df_display['ACTUAL_AS_OF'] = rev_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df_display[col] = rev_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2025036,2025-10-27 06:22:35,20250907,0,"2,732,299,409","1,029,837,825","604,293,363","425,544,462","1,292,459,988","770,779,671","521,680,317","370,142,141","39,859,455"
1,2025037,2025-10-27 06:22:35,20250914,"15,298,188","2,747,597,597","901,311,685","526,268,478","375,043,207","1,395,218,954","831,106,758","564,112,196","403,632,978","47,433,979"
2,2025038,2025-10-27 06:22:35,20250921,"-207,661,531","2,539,936,066","904,879,170","526,872,488","378,006,681","1,254,337,293","715,236,341","539,100,952","335,704,856","45,014,748"
3,2025039,2025-10-27 06:22:35,20250928,"523,123,575","3,063,059,641","895,748,085","526,844,688","368,903,397","1,599,231,716","1,034,099,333","565,132,383","500,497,244","67,582,597"
4,2025040,2025-10-27 06:22:35,20251005,"-290,767,647","2,772,291,995","1,104,285,279","652,473,170","451,812,108","1,334,586,311","714,337,029","620,249,282","222,128,759","111,291,646"
5,2025041,2025-10-27 06:22:35,20251012,"318,268,934","3,090,560,928","939,839,028","548,314,609","391,524,419","1,731,482,791","1,151,291,378","580,191,413","382,745,354","36,493,754"
6,2025042,2025-10-27 06:22:35,20251019,"611,425,521","3,701,986,449","936,284,717","546,060,979","390,223,737","2,196,328,427","1,657,462,820","538,865,607","510,648,779","58,724,527"
7,2025043,2025-10-27 06:22:35,20251025,"-1,711,572,688","1,990,413,762","797,790,373","462,681,778","335,108,595","780,485,572","546,686,241","464,534,669","164,050,013","17,352,466"


In [13]:
''' Revenue(Geo) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df['DoD'] = rev_geo_daily_df['TOTAL(Geo)'] - rev_geo_daily_df['TOTAL(Geo)'].shift()
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_daily_df_display = rev_geo_daily_df.copy()
mod_col_list = rev_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df_display[col] = rev_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202510,20251026,2025-10-27 06:22:35,"-209,969,123",0,0,0,0,0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,"3,090,724","209,969,123","132,898,275","77,741,515","55,156,760",0,0,"77,070,848",0,0
2,202510,20251024,2025-10-27 06:22:35,"7,662,323","206,878,398","130,189,624","75,532,147","54,657,477",0,0,"76,688,774",0,0
3,202510,20251023,2025-10-27 06:22:35,"-734,803,416","199,216,076","122,240,360","70,269,998","51,970,362",0,0,"76,975,715",0,0
4,202510,20251022,2025-10-27 06:22:35,"724,447,904","934,019,491","127,587,642","73,497,904","54,089,737","625,029,371","546,686,241","78,343,130","164,050,013","17,352,466"
5,202510,20251021,2025-10-27 06:22:35,"-21,187,500","209,571,587","131,687,372","75,510,061","56,177,311","77,884,215",0,"77,884,215",0,0
6,202510,20251020,2025-10-27 06:22:35,"-704,078,267","230,759,087","153,187,100","90,130,153","63,056,947","77,571,987",0,"77,571,987",0,0
7,202510,20251019,2025-10-27 06:22:35,"722,413,106","934,837,354","136,943,487","79,091,403","57,852,084","614,465,449","536,386,416","78,079,033","166,300,027","17,128,391"
8,202510,20251018,2025-10-27 06:22:35,"-3,389,432","212,424,248","135,110,232","78,160,140","56,950,092","77,314,016",0,"77,314,016",0,0
9,202510,20251017,2025-10-27 06:22:35,"-739,800,685","215,813,680","139,227,959","82,096,432","57,131,527","76,585,721",0,"76,585,721",0,0


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202509


### All Service

In [16]:
''' All Service (Corp) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_CD'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,All Services,B0R000100,Total Revenue,2025-10-27 02:25:40,"12,399,240,994","12,336,847,148",0,0,0,"62,393,846"
1,202509,All Services,DB0R000100,Total Revenue : DTAC,2025-10-27 02:25:40,"4,317,148,931","4,317,133,286",0,0,0,"15,645"
2,202509,All Services,TB0R000100,Total Revenue : TRUE,2025-10-27 02:25:40,"8,082,092,063","8,019,713,862",0,0,0,"62,378,201"


In [17]:
''' All Service (Geo) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_CD'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,All Services,B0R000100GEO,Total Revenue (Geo),2025-10-27 06:22:35,0,"11,756,719,123","11,756,636,791","11,752,642,524","11,752,128,722","-11,756,719,123"
1,202509,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-10-27 06:22:35,0,"4,136,061,858","4,136,060,461","4,135,973,591","4,135,973,591","-4,136,061,858"
2,202509,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-10-27 06:22:35,0,"7,620,657,265","7,620,576,330","7,616,668,933","7,616,155,131","-7,620,657,265"


### Mobile

In [18]:
''' Mobile (Corp) '''

v_product_grp = 'Mobile'

mobile_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[~mobile_df['METRIC_CD'].str.contains('GEO')]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Mobile,B0R000101,Mobile Revenue,2025-10-27 02:25:40,"10,454,801,824","10,450,736,078",0,0,0,"4,065,746"
1,202509,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-10-27 02:25:40,"4,317,148,931","4,317,133,286",0,0,0,"15,645"
2,202509,Mobile,TB0R000101,Mobile Revenue : TMH,2025-10-27 02:25:40,"6,137,652,893","6,133,602,792",0,0,0,"4,050,101"


In [19]:
''' Mobile (Geo) '''

v_product_grp = 'Mobile'

mobile_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['METRIC_CD'].str.contains('GEO')]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_geo_df = mobile_geo_df.reset_index(drop=True)
mobile_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-10-27 06:22:35,0,"9,870,608,053","9,870,525,721","9,870,427,272","9,870,427,272","-9,870,608,053"
1,202509,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-10-27 06:22:35,0,"4,136,061,858","4,136,060,461","4,135,973,591","4,135,973,591","-4,136,061,858"
2,202509,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-10-27 06:22:35,0,"5,734,546,195","5,734,465,260","5,734,453,681","5,734,453,681","-5,734,546,195"


### Prepaid

In [20]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Prepaid,B1R000100,Prepaid Revenue,2025-10-27 06:22:35,"4,060,781,813","4,060,781,813","4,060,706,210","4,060,618,201","4,060,618,201",0
1,202509,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"1,678,098,552","1,678,098,552","1,678,098,552","1,678,021,383","1,678,021,383",0
2,202509,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-10-27 06:22:35,"2,382,683,261","2,382,683,261","2,382,607,658","2,382,596,818","2,382,596,818",0


### Postpaid

In [21]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Postpaid,B2R000100,Postpaid Revenue,2025-10-27 02:25:40,"6,389,954,265","6,389,954,265",0,0,0,0
1,202509,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-10-27 06:22:35,"2,639,034,734","2,639,034,734","2,639,033,337","2,637,826,832","2,637,826,832",0
2,202509,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-10-27 02:25:40,"3,750,919,531","3,750,919,531",0,0,0,0


In [22]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Postpaid,B2R010100,Postpaid Revenue B2C,2025-10-27 06:22:35,"5,809,826,240","5,809,826,240","5,809,819,511","5,809,809,071","5,809,809,071",0
1,202509,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"2,457,963,306","2,457,963,306","2,457,961,909","2,457,952,208","2,457,952,208",0
2,202509,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,"3,351,862,934","3,351,862,934","3,351,857,602","3,351,856,863","3,351,856,863",0


In [23]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Postpaid,B2R020100,Postpaid Revenue B2B,2025-10-27 02:25:40,"580,126,826","580,126,826",0,0,0,0
1,202509,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-10-27 06:22:35,"181,071,428","181,071,428","181,071,428","179,874,624","179,874,624",0
2,202509,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-10-27 02:25:40,"399,055,398","399,055,398",0,0,0,0


### TOL

In [24]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,TOL,TB3R000100,TOL Revenue,2025-10-27 06:22:35,"1,609,977,219","1,609,977,219","1,609,977,219","1,609,977,219","1,609,977,219",0


### TVS

In [25]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,TVS,TB4R000100,TVS Revenue,2025-10-27 06:22:35,"276,133,851","276,133,851","276,133,851","272,238,033","271,724,231",0


## ** Special Metric (Corp & Geo)

### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [26]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202401,2025-09-22 06:15:44,"33,068,240","12,494,707,100","12,527,775,340","2,211,126,641","1,803,547,883","3,722,556,455","2,911,240,546","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,"23,506,585","12,224,536,694","12,248,043,279","2,082,104,053","1,714,609,789","3,686,285,226","2,886,648,865","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,"25,129,495","12,459,268,326","12,484,397,821","2,226,380,294","1,804,408,125","3,658,095,837","2,888,375,734","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,"23,870,945","12,334,721,853","12,358,592,798","2,140,030,223","1,743,154,862","3,692,736,895","2,880,091,836","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,"25,540,442","12,440,265,912","12,465,806,354","2,269,268,086","1,773,106,846","3,675,457,906","2,844,558,832","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,"23,747,283","12,347,976,614","12,371,723,897","2,206,174,307","1,726,295,968","3,712,964,588","2,824,759,226","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,"26,564,154","12,463,315,685","12,489,879,839","2,272,833,725","1,799,831,136","3,667,787,168","2,817,043,356","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,"29,588,466","12,594,999,452","12,624,587,918","2,312,752,271","1,797,206,658","3,680,722,062","2,802,327,404","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,"30,443,177","12,479,579,804","12,510,022,981","2,245,163,825","1,737,197,388","3,706,292,256","2,784,939,778","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,"45,904,631","12,523,605,918","12,569,510,549","2,345,529,988","1,782,644,560","3,685,237,458","2,800,494,445","1,598,256,224","311,443,243"


In [27]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202401,2025-09-22 06:15:44,"33,180,744","7,779,918,671","7,813,099,416","2,211,126,641","3,722,556,455","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,"23,617,016","7,623,278,040","7,646,895,055","2,082,104,053","3,686,285,226","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,"25,216,744","7,766,484,467","7,791,701,211","2,226,380,294","3,658,095,837","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,"23,962,132","7,711,475,155","7,735,437,287","2,140,030,223","3,692,736,895","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,"25,620,601","7,822,600,234","7,848,220,835","2,269,268,086","3,675,457,906","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,"23,832,682","7,796,921,421","7,820,754,103","2,206,174,307","3,712,964,588","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,"26,666,256","7,846,441,194","7,873,107,450","2,272,833,725","3,667,787,168","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,"29,693,737","7,995,465,389","8,025,159,126","2,312,752,271","3,680,722,062","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,"30,526,907","7,957,442,638","7,987,969,545","2,245,163,825","3,706,292,256","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,"45,999,504","7,940,466,913","7,986,466,416","2,345,529,988","3,685,237,458","1,598,256,224","311,443,243"


In [28]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202401,2025-09-22 06:15:44,"-112,505","4,714,788,429","4,714,675,924","1,803,547,883","2,911,240,546"
1,202402,2025-09-22 06:15:44,"-110,430","4,601,258,654","4,601,148,224","1,714,609,789","2,886,648,865"
2,202403,2025-09-22 06:15:44,"-87,249","4,692,783,860","4,692,696,611","1,804,408,125","2,888,375,734"
3,202404,2025-09-22 06:15:44,"-91,187","4,623,246,698","4,623,155,511","1,743,154,862","2,880,091,836"
4,202405,2025-09-22 06:15:44,"-80,159","4,617,665,679","4,617,585,519","1,773,106,846","2,844,558,832"
5,202406,2025-09-22 06:15:44,"-85,399","4,551,055,193","4,550,969,794","1,726,295,968","2,824,759,226"
6,202407,2025-09-22 06:15:44,"-102,102","4,616,874,491","4,616,772,389","1,799,831,136","2,817,043,356"
7,202408,2025-09-22 06:15:44,"-105,271","4,599,534,063","4,599,428,792","1,797,206,658","2,802,327,404"
8,202409,2025-09-22 06:15:44,"-83,730","4,522,137,166","4,522,053,436","1,737,197,388","2,784,939,778"
9,202410,2025-09-22 06:15:44,"-94,873","4,583,139,005","4,583,044,133","1,782,644,560","2,800,494,445"


In [29]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202401,2025-09-22 06:15:44,"88,428","10,648,471,525","10,648,559,952","2,211,126,641","1,803,547,883","3,722,556,455","2,911,240,546"
1,202402,2025-09-22 06:15:44,"273,318","10,369,647,933","10,369,921,251","2,082,104,053","1,714,609,789","3,686,285,226","2,886,648,865"
2,202403,2025-09-22 06:15:44,"555,550","10,577,259,991","10,577,815,541","2,226,380,294","1,804,408,125","3,658,095,837","2,888,375,734"
3,202404,2025-09-22 06:15:44,"1,184,927","10,456,013,816","10,457,198,744","2,140,030,223","1,743,154,862","3,692,736,895","2,880,091,836"
4,202405,2025-09-22 06:15:44,"4,094,170","10,562,391,671","10,566,485,841","2,269,268,086","1,773,106,846","3,675,457,906","2,844,558,832"
5,202406,2025-09-22 06:15:44,"4,013,650","10,470,194,088","10,474,207,738","2,206,174,307","1,726,295,968","3,712,964,588","2,824,759,226"
6,202407,2025-09-22 06:15:44,"4,676,194","10,557,495,385","10,562,171,579","2,272,833,725","1,799,831,136","3,667,787,168","2,817,043,356"
7,202408,2025-09-22 06:15:44,"4,177,481","10,593,008,396","10,597,185,877","2,312,752,271","1,797,206,658","3,680,722,062","2,802,327,404"
8,202409,2025-09-22 06:15:44,"2,880,349","10,473,593,247","10,476,473,596","2,245,163,825","1,737,197,388","3,706,292,256","2,784,939,778"
9,202410,2025-09-22 06:15:44,"3,753,058","10,613,906,451","10,617,659,509","2,345,529,988","1,782,644,560","3,685,237,458","2,800,494,445"


In [30]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202401,2025-09-22 06:15:44,"200,932","5,933,683,096","5,933,884,028","2,211,126,641","3,722,556,455"
1,202402,2025-09-22 06:15:44,"383,748","5,768,389,279","5,768,773,028","2,082,104,053","3,686,285,226"
2,202403,2025-09-22 06:15:44,"642,799","5,884,476,131","5,885,118,930","2,226,380,294","3,658,095,837"
3,202404,2025-09-22 06:15:44,"1,276,114","5,832,767,118","5,834,043,232","2,140,030,223","3,692,736,895"
4,202405,2025-09-22 06:15:44,"4,174,329","5,944,725,992","5,948,900,322","2,269,268,086","3,675,457,906"
5,202406,2025-09-22 06:15:44,"4,099,049","5,919,138,895","5,923,237,944","2,206,174,307","3,712,964,588"
6,202407,2025-09-22 06:15:44,"4,778,296","5,940,620,894","5,945,399,190","2,272,833,725","3,667,787,168"
7,202408,2025-09-22 06:15:44,"4,282,752","5,993,474,334","5,997,757,085","2,312,752,271","3,680,722,062"
8,202409,2025-09-22 06:15:44,"2,964,079","5,951,456,081","5,954,420,160","2,245,163,825","3,706,292,256"
9,202410,2025-09-22 06:15:44,"3,847,930","6,030,767,446","6,034,615,376","2,345,529,988","3,685,237,458"


In [31]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202401,2025-09-22 06:15:44,"-112,505","4,714,788,429","4,714,675,924","1,803,547,883","2,911,240,546"
1,202402,2025-09-22 06:15:44,"-110,430","4,601,258,654","4,601,148,224","1,714,609,789","2,886,648,865"
2,202403,2025-09-22 06:15:44,"-87,249","4,692,783,860","4,692,696,611","1,804,408,125","2,888,375,734"
3,202404,2025-09-22 06:15:44,"-91,187","4,623,246,698","4,623,155,511","1,743,154,862","2,880,091,836"
4,202405,2025-09-22 06:15:44,"-80,159","4,617,665,679","4,617,585,519","1,773,106,846","2,844,558,832"
5,202406,2025-09-22 06:15:44,"-85,399","4,551,055,193","4,550,969,794","1,726,295,968","2,824,759,226"
6,202407,2025-09-22 06:15:44,"-102,102","4,616,874,491","4,616,772,389","1,799,831,136","2,817,043,356"
7,202408,2025-09-22 06:15:44,"-105,271","4,599,534,063","4,599,428,792","1,797,206,658","2,802,327,404"
8,202409,2025-09-22 06:15:44,"-83,730","4,522,137,166","4,522,053,436","1,737,197,388","2,784,939,778"
9,202410,2025-09-22 06:15:44,"-94,873","4,583,139,005","4,583,044,133","1,782,644,560","2,800,494,445"


### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [32]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-09-22 06:15:44,0,"11,881,730,399","11,881,730,399","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,0,"11,626,563,733","11,626,563,733","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,0,"11,853,639,343","11,853,639,343","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,0,"11,723,479,404","11,723,479,404","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,0,"11,841,867,846","11,841,867,846","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,0,"11,719,185,282","11,719,185,282","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,0,"11,857,212,955","11,857,212,955","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,-0,"11,992,892,504","11,992,892,504","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,0,"11,856,710,578","11,856,710,578","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,0,"11,900,384,524","11,900,384,524","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [33]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202401,2025-09-22 06:15:44,0,"7,368,644,248","7,368,644,248","2,211,126,641","3,311,282,032","1,507,069,397","339,166,179"
1,202402,2025-09-22 06:15:44,0,"7,222,065,281","7,222,065,281","2,082,104,053","3,285,072,468","1,523,039,176","331,849,584"
2,202403,2025-09-22 06:15:44,0,"7,362,485,852","7,362,485,852","2,226,380,294","3,254,097,222","1,539,811,431","342,196,905"
3,202404,2025-09-22 06:15:44,0,"7,298,917,238","7,298,917,238","2,140,030,223","3,280,178,978","1,546,917,278","331,790,759"
4,202405,2025-09-22 06:15:44,0,"7,420,456,954","7,420,456,954","2,269,268,086","3,273,314,626","1,554,971,583","322,902,658"
5,202406,2025-09-22 06:15:44,0,"7,368,914,105","7,368,914,105","2,206,174,307","3,284,957,272","1,563,892,736","313,889,791"
6,202407,2025-09-22 06:15:44,0,"7,447,886,733","7,447,886,733","2,272,833,725","3,269,232,707","1,577,812,706","328,007,594"
7,202408,2025-09-22 06:15:44,0,"7,598,058,116","7,598,058,116","2,312,752,271","3,283,314,789","1,592,482,287","409,508,769"
8,202409,2025-09-22 06:15:44,0,"7,532,631,092","7,532,631,092","2,245,163,825","3,281,480,710","1,633,490,490","372,496,067"
9,202410,2025-09-22 06:15:44,-0,"7,527,416,008","7,527,416,008","2,345,529,988","3,272,186,553","1,598,256,224","311,443,243"


In [34]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202401,2025-09-22 06:15:44,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-09-22 06:15:44,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-09-22 06:15:44,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-09-22 06:15:44,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-09-22 06:15:44,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-09-22 06:15:44,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-09-22 06:15:44,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-09-22 06:15:44,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-09-22 06:15:44,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-09-22 06:15:44,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


In [35]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202401,2025-09-22 06:15:44,0,"10,035,494,824","10,035,494,824","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268"
1,202402,2025-09-22 06:15:44,0,"9,771,674,973","9,771,674,973","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663"
2,202403,2025-09-22 06:15:44,0,"9,971,631,007","9,971,631,007","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365"
3,202404,2025-09-22 06:15:44,0,"9,844,771,366","9,844,771,366","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303"
4,202405,2025-09-22 06:15:44,-0,"9,963,993,605","9,963,993,605","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046"
5,202406,2025-09-22 06:15:44,0,"9,841,402,756","9,841,402,756","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210"
6,202407,2025-09-22 06:15:44,0,"9,951,392,655","9,951,392,655","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087"
7,202408,2025-09-22 06:15:44,0,"9,990,901,448","9,990,901,448","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729"
8,202409,2025-09-22 06:15:44,0,"9,850,724,021","9,850,724,021","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098"
9,202410,2025-09-22 06:15:44,0,"9,990,685,057","9,990,685,057","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955"


In [36]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202401,2025-09-22 06:15:44,0,"5,522,408,673","5,522,408,673","2,211,126,641","3,311,282,032"
1,202402,2025-09-22 06:15:44,-0,"5,367,176,521","5,367,176,521","2,082,104,053","3,285,072,468"
2,202403,2025-09-22 06:15:44,0,"5,480,477,516","5,480,477,516","2,226,380,294","3,254,097,222"
3,202404,2025-09-22 06:15:44,0,"5,420,209,201","5,420,209,201","2,140,030,223","3,280,178,978"
4,202405,2025-09-22 06:15:44,-0,"5,542,582,712","5,542,582,712","2,269,268,086","3,273,314,626"
5,202406,2025-09-22 06:15:44,0,"5,491,131,578","5,491,131,578","2,206,174,307","3,284,957,272"
6,202407,2025-09-22 06:15:44,0,"5,542,066,432","5,542,066,432","2,272,833,725","3,269,232,707"
7,202408,2025-09-22 06:15:44,0,"5,596,067,060","5,596,067,060","2,312,752,271","3,283,314,789"
8,202409,2025-09-22 06:15:44,0,"5,526,644,535","5,526,644,535","2,245,163,825","3,281,480,710"
9,202410,2025-09-22 06:15:44,0,"5,617,716,541","5,617,716,541","2,345,529,988","3,272,186,553"


In [37]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202401,2025-09-22 06:15:44,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-09-22 06:15:44,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-09-22 06:15:44,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-09-22 06:15:44,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-09-22 06:15:44,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-09-22 06:15:44,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-09-22 06:15:44,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-09-22 06:15:44,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-09-22 06:15:44,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-09-22 06:15:44,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


## ** Issue History

In [38]:
''' Issue : Postpaid Revenue : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R000100'

issue_postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.loc[issue_postpaid_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,722,556,455","3,722,556,455",0,0,0,0
1,202402,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,686,285,226","3,686,285,226",0,0,0,0
2,202403,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,658,095,837","3,658,095,837",0,0,0,0
3,202404,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,692,736,895","3,692,736,895",0,0,0,0
4,202405,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,675,457,906","3,675,457,906",0,0,0,0
5,202406,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,712,964,588","3,712,964,588",0,0,0,0
6,202407,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,667,787,168","3,667,787,168",0,0,0,0
7,202408,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,680,722,062","3,680,722,062",0,0,0,0
8,202409,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,706,292,256","3,706,292,256",0,0,0,0
9,202410,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-09-22 02:25:18,"3,685,237,458","3,685,237,458",0,0,0,0


In [39]:
''' Issue : Postpaid Revenue B2C : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R010100'

issue_postpaid_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.loc[issue_postpaid_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,311,282,032","3,311,282,032","3,305,512,181","3,305,485,985","3,305,485,985",0
1,202402,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,285,072,468","3,285,072,468","3,279,470,827","3,279,449,587","3,279,449,587",0
2,202403,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,254,097,222","3,254,097,222","3,248,644,556","3,248,622,787","3,248,622,787",0
3,202404,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,280,178,978","3,280,178,978","3,274,715,195","3,274,699,228","3,274,699,228",0
4,202405,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,273,314,626","3,273,314,626","3,267,800,934","3,267,780,094","3,267,780,094",0
5,202406,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,284,957,272","3,284,957,272","3,278,609,892","3,278,595,142","3,278,595,142",0
6,202407,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,269,232,707","3,269,232,707","3,264,050,341","3,264,050,142","3,264,050,142",0
7,202408,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,283,314,789","3,283,314,789","3,278,134,554","3,278,134,355","3,278,134,355",0
8,202409,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,281,480,710","3,281,480,710","3,276,410,628","3,276,410,449","3,276,410,449",0
9,202410,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-09-22 06:15:44,"3,272,186,553","3,272,186,553","3,266,340,594","3,266,340,415","3,266,340,415",0


In [40]:
''' Issue : Postpaid Revenue B2B : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R020100'

issue_postpaid_b2b_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.loc[issue_postpaid_b2b_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2b_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"411,274,423","411,274,423",0,0,0,0
1,202402,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"401,212,758","401,212,758",0,0,0,0
2,202403,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"403,998,615","403,998,615",0,0,0,0
3,202404,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"412,557,917","412,557,917",0,0,0,0
4,202405,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"402,143,280","402,143,280",0,0,0,0
5,202406,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"428,007,316","428,007,316",0,0,0,0
6,202407,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"398,554,461","398,554,461",0,0,0,0
7,202408,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"397,407,274","397,407,274",0,0,0,0
8,202409,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"424,811,546","424,811,546",0,0,0,0
9,202410,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-09-22 02:25:18,"413,050,904","413,050,904",0,0,0,0


In [41]:
''' Daily : Postpaid Revenue B2C : TMH '''

v_metict_cd = 'TB2R010100'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rev_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.loc[issue_daily_post_rev_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rev_b2c_tmh_df_display = issue_daily_post_rev_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_rev_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rev_b2c_tmh_df_display[col] = issue_daily_post_rev_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rev_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
2,202510,20251024,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
3,202510,20251023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
4,202510,20251022,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,"546,686,241","546,685,383","546,685,343","546,685,343"
5,202510,20251021,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
6,202510,20251020,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
7,202510,20251019,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,"536,386,416","536,385,702","536,385,702","536,385,702"
8,202510,20251018,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
9,202510,20251017,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan


## ** Current Issue

In [42]:
''' Daily : Postpaid Revenue B2C : DTAC '''

v_metict_cd = 'DB2R010100'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rev_b2c_dtac_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rev_b2c_dtac_df = issue_daily_post_rev_b2c_dtac_df.loc[issue_daily_post_rev_b2c_dtac_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rev_b2c_dtac_df = issue_daily_post_rev_b2c_dtac_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rev_b2c_dtac_df_display = issue_daily_post_rev_b2c_dtac_df.copy()
mod_col_list = issue_daily_post_rev_b2c_dtac_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rev_b2c_dtac_df_display[col] = issue_daily_post_rev_b2c_dtac_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rev_b2c_dtac_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"77,070,848","77,070,848","77,069,829","77,069,829"
2,202510,20251024,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"76,688,774","76,688,774","76,688,774","76,688,774"
3,202510,20251023,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"76,975,715","76,975,216","76,975,198","76,975,198"
4,202510,20251022,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"78,343,130","78,342,631","78,342,293","78,342,293"
5,202510,20251021,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"77,884,215","77,884,215","77,883,756","77,883,756"
6,202510,20251020,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"77,571,987","77,571,588","77,571,588","77,571,588"
7,202510,20251019,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"78,079,033","78,079,033","78,078,334","78,078,334"
8,202510,20251018,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"77,314,016","77,314,016","77,313,497","77,313,497"
9,202510,20251017,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-10-27 06:22:35,"76,585,721","76,585,721","76,585,721","76,585,721"


In [43]:
''' Daily : Prepaid Revenue : DTAC '''

v_metict_cd = 'DB1R000100'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_pre_rev_dtac_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_pre_rev_dtac_df = issue_daily_pre_rev_dtac_df.loc[issue_daily_pre_rev_dtac_df['METRIC_CD']==v_metict_cd]
issue_daily_pre_rev_dtac_df = issue_daily_pre_rev_dtac_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_pre_rev_dtac_df_display = issue_daily_pre_rev_dtac_df.copy()
mod_col_list = issue_daily_pre_rev_dtac_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_pre_rev_dtac_df_display[col] = issue_daily_pre_rev_dtac_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_pre_rev_dtac_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"55,156,760","55,156,760","55,156,134","55,156,134"
2,202510,20251024,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"54,657,477","54,657,477","54,656,445","54,656,445"
3,202510,20251023,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"51,970,362","51,970,362","51,970,027","51,970,027"
4,202510,20251022,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"54,089,737","54,089,737","54,087,831","54,087,831"
5,202510,20251021,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"56,177,311","56,177,311","56,172,454","56,172,454"
6,202510,20251020,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"63,056,947","63,056,947","63,055,233","63,055,233"
7,202510,20251019,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"57,852,084","57,852,084","57,851,052","57,851,052"
8,202510,20251018,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"56,950,092","56,950,092","56,944,815","56,944,815"
9,202510,20251017,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-10-27 06:22:35,"57,131,527","57,131,527","57,129,342","57,129,342"
